# 1. Set up Pinecone & Langchain

## 1.1 Credentials - get API key

In [3]:
import os
from getpass import getpass

from pinecone import Pinecone
import langchain

In [4]:
# VectorDB
import getpass # For secure password input
import os

from pinecone import Pinecone

os.environ["PINECONE_API_KEY"] = os.getenv("PINECONE_API_KEY") or getpass(
    "Enter your Pinecone API key: "
)

pinecone_api_key = os.getenv("PINECONE_API_KEY")

pc = Pinecone(api_key=pinecone_api_key)

## 1.2 Initialization
### 1.2.1 Check for index - create index

Index is a workspace inside of Pinecone DB - a namespace is a folder inside an index

In [5]:
index_name = "llama-text-embed-v2"
if not pc.has_index(index_name):
    pc.create_index_for_model(
        name=index_name,
        cloud="aws",
        region="us-east-1",
        embed={
            "model": "llama-text-embed-v2",
            "field_map": {"text": "text"}
        }
    )
index = pc.Index(index_name)

### 1.2.2 Create Langchain Wrapper for Pinecone Embedding model

Sources:
- [Langchain Huggingface Docs](https://python.langchain.com/api_reference/huggingface/embeddings/langchain_huggingface.embeddings.huggingface.HuggingFaceEmbeddings.html)
- [Langchain Pinecone Embeddings Docs](https://python.langchain.com/docs/integrations/text_embedding/pinecone/)
- [Langchain Pinecone Docs](https://python.langchain.com/docs/integrations/vectorstores/pinecone/#initialization)
- [Pinecone llama-text-embed-v2 docs](https://docs.pinecone.io/models/llama-text-embed-v2)

In [6]:
from langchain_pinecone import PineconeEmbeddings

embeddings = PineconeEmbeddings(model="llama-text-embed-v2")

### 1.2.3 Init Langchain Vectorstore

In [7]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

delete all data in db before adding new facts & rules

In [8]:
vector_store._index.delete(delete_all=True)

{}

Every Document has:
- .page_content → the text
- .metadata → a dictionary

Note:

Facts & Rules sind ChatGPT generiert mit folgendem Prompt:

"give me 5 facts and 5 rules about siemens and a prompt for testing"

In [10]:
from uuid import uuid4

from langchain_core.documents import Document

# --- Facts ---
document_1 = Document(
    page_content="Siemens was founded in 1847 by Werner von Siemens in Berlin, Germany.",
    metadata={"source": "facts"},
)

document_2 = Document(
    page_content="Siemens is a global leader in electrification, automation, and digitalization solutions.",
    metadata={"source": "facts"},
)

document_3 = Document(
    page_content="Siemens operates in more than 190 countries worldwide.",
    metadata={"source": "facts"},
)

document_4 = Document(
    page_content="Siemens Energy, a spin-off from Siemens AG, focuses on power generation and transmission.",
    metadata={"source": "facts"},
)

document_5 = Document(
    page_content="The Siemens logo is one of the most recognizable industrial logos in the world.",
    metadata={"source": "facts"},
)

# --- Rules ---
document_6 = Document(
    page_content="If a company provides automation and digitalization solutions, then it supports Industry 4.0.",
    metadata={"source": "rules"},
)

document_7 = Document(
    page_content="If a firm has operations in more than 100 countries, then it is considered a multinational corporation.",
    metadata={"source": "rules"},
)

document_8 = Document(
    page_content="If a business is a leader in electrification, then it contributes to energy transition and grid modernization.",
    metadata={"source": "rules"},
)

document_9 = Document(
    page_content="If a company spins off a specialized division, then that division operates as an independent entity.",
    metadata={"source": "rules"},
)

document_10 = Document(
    page_content="If a brand is widely recognized worldwide, then it has strong global brand equity.",
    metadata={"source": "rules"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]
uuids = [str(uuid4()) for _ in range(len(documents))]
vector_store.add_documents(documents=documents, ids=uuids)

['5c32ca0b-d89b-4a10-8ed4-170926c2d179',
 '727edef1-dfec-4c76-ba15-a8399e1f331d',
 '6ef3f4bb-ce78-44c4-a8a0-d5066a10f70a',
 '4c56f184-8f1f-4677-b485-a4139b08b8df',
 '075d0fcb-fbfa-4b60-8f71-7889a9c8e21c',
 '959bf3be-3e5e-4efe-8208-a57bc54eda59',
 '84652daa-e939-45fd-85f2-b630f82ae43b',
 'a96f04c4-bf91-47aa-bdcf-5a2cc2ec3eb4',
 'd803546d-f938-430c-b000-f07bf55b0477',
 'e0118c35-9f20-4b83-810b-9de20cbc4076']

In [12]:
prompt = "Why is Siemens considered an important player in the global energy transition?"

### 1.2.4 Test Data & Test code - Ignore whole SECTION while run

In [32]:
# docs = [
#     "Apple is a popular fruit known for its sweetness and crisp texture.",
#     "The tech company Apple is known for its innovative products like the iPhone.",
#     "Many people enjoy eating apples as a healthy snack.",
#     "Apple Inc. has revolutionized the tech industry with its sleek designs and user-friendly interfaces.",
#     "An apple a day keeps the doctor away, as the saying goes.",
# ]

In [33]:
# doc_embeds = embeddings.embed_documents(docs)
# doc_embeds

In [34]:
# query = "Tell me about the tech company known as Apple"
# query_embed = embeddings.embed_query(query)
# query_embed

### 1.2.5 Similarity Search
#### 1.2.5.1 Test - Ignore while run

In [35]:
# results = vector_store.similarity_search(
#     "LangChain provides abstractions to make working with LLMs easy",
#     k=2,
#     filter={"source": "tweet"},
# )
# for res in results:
#     print(f"* {res.page_content} [{res.metadata}]")

#### 1.2.5.2 Implementation

In [13]:
relevant_docs = vector_store.similarity_search(
    prompt,
    k = 4,
)

In [14]:
print("Relevant documents found:", relevant_docs)

Relevant documents found: [Document(id='727edef1-dfec-4c76-ba15-a8399e1f331d', metadata={'source': 'facts'}, page_content='Siemens is a global leader in electrification, automation, and digitalization solutions.'), Document(id='0660ae8d-eb27-44f8-aa3f-4a3f64306794', metadata={'source': 'facts'}, page_content='Siemens is a global leader in electrification, automation, and digitalization solutions.'), Document(id='a96f04c4-bf91-47aa-bdcf-5a2cc2ec3eb4', metadata={'source': 'rules'}, page_content='If a business is a leader in electrification, then it contributes to energy transition and grid modernization.'), Document(id='160a794e-fd73-4c6a-ba6b-2ca27b7a0ebe', metadata={'source': 'rules'}, page_content='If a business is a leader in electrification, then it contributes to energy transition and grid modernization.')]


In [15]:
facts = [doc for doc in relevant_docs if doc.metadata.get("source") == "facts"]
rules = [doc for doc in relevant_docs if doc.metadata.get("source") == "rules"]

In [16]:
facts_str = "\n".join([f"- {doc.page_content}" for doc in facts])
rules_str = "\n".join([f"- {doc.page_content}" for doc in rules])

# 2. Gemini LLM

## 2.1 Init Gemini LLM
Sources:
- [Langchain LLM](https://python.langchain.com/api_reference/core/language_models/langchain_core.language_models.llms.LLM.html)
- [Google AI Langchain LLM](https://python.langchain.com/docs/integrations/providers/google/#llms)

In [17]:
from langchain_google_genai import GoogleGenerativeAI

llm = GoogleGenerativeAI(
    model="gemini-2.5-flash",
    google_api_key= os.getenv("GOOGLE_API_KEY")
    )

E0000 00:00:1758757324.849893 8107704 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


## 2.2 Prompt Template
Sources:
- [Prompt Template LC](https://python.langchain.com/docs/concepts/prompt_templates/)

In [19]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    """Based only on the following retrieved relevant facts:
        {facts}
    
        and rules:
        {rules}

        answer the following question:
        {prompt}
    """
)

message = prompt_template.invoke({
    "facts": facts_str,
    "rules": rules_str,
    "prompt": prompt
    })

In [20]:
response = llm.invoke(message)

In [21]:
print(relevant_docs, "\n")
print("Facts used:\n", facts_str, "\n")
print("Rules used:\n", rules_str, "\n")
print("Prompt:\n", prompt, "\n")

print("Final Response from LLM:", response)

[Document(id='727edef1-dfec-4c76-ba15-a8399e1f331d', metadata={'source': 'facts'}, page_content='Siemens is a global leader in electrification, automation, and digitalization solutions.'), Document(id='0660ae8d-eb27-44f8-aa3f-4a3f64306794', metadata={'source': 'facts'}, page_content='Siemens is a global leader in electrification, automation, and digitalization solutions.'), Document(id='a96f04c4-bf91-47aa-bdcf-5a2cc2ec3eb4', metadata={'source': 'rules'}, page_content='If a business is a leader in electrification, then it contributes to energy transition and grid modernization.'), Document(id='160a794e-fd73-4c6a-ba6b-2ca27b7a0ebe', metadata={'source': 'rules'}, page_content='If a business is a leader in electrification, then it contributes to energy transition and grid modernization.')] 

Facts used:
 - Siemens is a global leader in electrification, automation, and digitalization solutions.
- Siemens is a global leader in electrification, automation, and digitalization solutions. 

Rule